# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [1]:
install.packages("httr")
install.packages("tokenizers")
install.packages("stringr")

library(httr)
library(tokenizers)
library(stringr)

tokenize_text <- function(text) {
  tokenizers::tokenize_words(text, lowercase=TRUE, strip_punct=TRUE)[[1]]
}

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

also installing the dependency ‘SnowballC’


Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)



#### b) Make a function generate keys for ngrams.

In [2]:
key_from <- function(ngram, sep = "\x1f") {
  paste(ngram, collapse=sep)
}

#### c) Make a function to build an ngram table.

In [3]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
  if (length(tokens) < n) return(new.env(parent = emptyenv()))
  tbl <- new.env(parent = emptyenv())
  for (i in seq_len(length(tokens) - n + 1L)) {
    ngram <- tokens[i:(i + n - 2L)]
    next_word <- tokens[i + n - 1L]
    key <- paste(ngram, collapse = sep)
    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (next_word %in% names(counts)) {
      counts[[next_word]] <- counts[[next_word]] + 1L
    } else {
      counts[[next_word]] <- 1L
    }
    tbl[[key]] <- counts
  }
  tbl
}

#### d) Function to digest the text.

In [4]:
digest_text <- function(text, n) {
  tokens <- tokenize_text(text)
  build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [5]:
digest_url <- function(url, n) {
  res <- httr::GET(url)
  txt <- httr::content(res, as = "text", encoding = "UTF-8")
  digest_text(txt,n)
}

#### f) Function that gives random start.

In [6]:
random_start <- function(tbl, sep = "\x1f") {
  keys <- ls(envir = tbl, all.names=TRUE)
  if (length(keys)==0) stop("No n-grams available. Digest text first.")
  picked <- sample(keys, 1)
  strsplit(picked, sep, fixed=TRUE)[[1]]
}

#### g) Function to predict the next word.

In [7]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
  key <- paste(ngram, collapse = sep)
  counts <- if(!is.null(tbl[[key]])) tbl[[key]] else integer(0)
  if (length(counts) == 0) return(NA_character_)
  sample(names(counts), size=1, prob=as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [8]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
  force(tbl); n <- as.integer(n); force(sep)
  function(start_words = NULL, length = 10L) {
    if ((is.null(start_words)) || length(start_words) != n - 1L) {
      start_words <- random_start(tbl, sep=sep)
    }
    word_sequence <- start_words
    for (i in seq_len(max(0L, length - length(start_words)))) {
      ngram <- tail(word_sequence, n - 1L)
      next_word <- predict_next_word(tbl, ngram, sep=sep)
      if (is.na(next_word)) break
      word_sequence <- c(word_sequence, next_word)
    }
    paste(word_sequence, collapse= " ")
  }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [9]:
#a)
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)

In [13]:
#i)
set.seed(2025)  
output_i <- gen3(start_words = c("the", "king"), length = 15)
cat("i) Start words = 'the king'\n", output_i, "\n")

i) Start words = 'the king'
 the king has forbidden me to marry another husband am not i shall ride upon 


In [ ]:
# "the king has forbidden me to marry another husband am not i shall ride upon"

In [14]:
#ii)
set.seed(123)  
output_ii <- gen3(length = 15)
cat("\nii) Random start\n", output_ii, "\n")


ii) Random start
 a rapid stream how shall i kill her on a sudden looked behind the other 


In [ ]:
# "a rapid stream how shall i kill her on a sudden looked behind the other"

#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [15]:
#b)
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)

In [16]:
#i)
set.seed(2025)  
output_i <- gen3(start_words = c("the", "king"), length = 15)
cat("i) Start words = 'the king'\n", output_i, "\n")

i) Start words = 'the king'
 the king he added to the entire exclusion of the swords were made prisoners the 


In [ ]:
# "the king he added to the entire exclusion of the swords were made prisoners the"

In [17]:
#ii)
set.seed(2025) 
output_ii <- gen3(length = 15)
cat("\nii) Random start\n", output_ii, "\n")


ii) Random start
 lamentation de lemburn came forth completely armed after the fashion of this may be seen 


In [18]:
# "lamentation de lemburn came forth completely armed after the fashion of this may be seen"

#### c) Explain in 1-2 sentences the difference in content generated from each source.

The sentence in 2b draws on words more used in fairy tales (marry, husband, stream), while the sentence in 2c draws on more military or monarchy-related terms (exclusion, swords, prisoners, armed). This difference in content generated is due to the different bodies of text from which we sourced.

## Question 3
#### a) What is a language learning model? 

A language learning model is a tool that identifies the probability of the distribution of words and uses that probability to predict the next word in a sentence based on what came before.


#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

OLLAMA, a wrapper around Docker, is what can run a language model locally in the event that the internet is down. 


## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | A shell exists around an operating system but lets you interact with it. |
| **Terminal emulator** | A terminal emulator hosts a shell, providing the interface. |
| **Process** | What is running. It communicates using standard input/standard output. |
| **Signal** | What is sent to a process telling it to do something. |
| **Standard input** | A characteristic of a process; it reads characters in. |
| **Standard output** | A characteristic of a process; what the process reads in is fed to the standard output. |
| **Command line argument** | These arguments are passed to processes. |
| **The environment** | What a process can "see" while running. |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?

The programs are find, xargs, and grep

#### b) Explain what this command is doing, part by part.

This line is: searching the current directory; finding files ending in ".R"; thensearching within identified files for the use of "read_csv."

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.

I would see "Hello from Docker!"

#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.

The input would be docker run -it -e PASSWORD=<(mypassword)> -p 8787:8787 rocker/verse; the output would be the password I set.

#### c) How do you log in to the RStudio server?

I would go to  http://localhost:8787 and enter the username rstudio and the password I set up.